In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from itertools import combinations
from sklearn.metrics import mean_squared_error

# Генерация случайных данных
np.random.seed(42)
n_samples = 20
X = np.random.rand(n_samples, 5)  # 5 входных переменных
Y = 3 * X[:, 0] + 2 * X[:, 1] - X[:, 2] + np.random.normal(0, 0.1, n_samples)  # Истинная зависимость

# Разделение данных на обучающую и экзаменационную выборки
train_size = int(0.7 * n_samples)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Функция для перебора всех возможных моделей и выбора лучшей по критерию минимального смещения
def mgua_model_selection(X_train, Y_train, X_test, Y_test):
    best_model = None
    best_score = float('inf')
    best_features = None
    results = []

    num_features = X_train.shape[1]
    feature_indices = range(num_features)

    for r in range(1, num_features + 1):  # Перебираем модели с разным количеством переменных
        for feature_subset in combinations(feature_indices, r):
            X_train_subset = X_train[:, feature_subset]
            X_test_subset = X_test[:, feature_subset]

            model = LinearRegression()
            model.fit(X_train_subset, Y_train)

            Y_pred = model.predict(X_test_subset)
            error = mean_squared_error(Y_test, Y_pred)  # Критерий минимального смещения

            results.append({
                'Используемые признаки': str(feature_subset),
                'MSE': round(error, 5)
            })

            if error < best_score:
                best_score = error
                best_model = model
                best_features = feature_subset

    return best_model, best_features, pd.DataFrame(results)

# Запуск алгоритма
best_model, best_features, results_df = mgua_model_selection(X_train, Y_train, X_test, Y_test)

# Создание DataFrame для вывода в формате таблицы
results_df_sorted = results_df.sort_values(by='MSE')
results_df_sorted.to_csv("results.csv", index=False, encoding="utf-8")
results_df_sorted.to_excel("results.xlsx", index=False, engine="openpyxl")
print("Оптимальная модель использует признаки:", best_features)
print("Минимальная ошибка:", results_df_sorted.iloc[0]['MSE'])

# Формируем DataFrame в виде таблицы
results_df_sorted.columns = ['Используемые признаки', 'Среднеквадратичная ошибка (MSE)']
print("\nТаблица результатов:")
print(results_df_sorted.to_markdown(index=False))



Оптимальная модель использует признаки: (0, 1, 2, 3)
Минимальная ошибка: 0.00525

Таблица результатов:
| Используемые признаки   |   Среднеквадратичная ошибка (MSE) |
|:------------------------|----------------------------------:|
| (0, 1, 2, 3)            |                           0.00525 |
| (0, 1, 2)               |                           0.00652 |
| (0, 1, 2, 4)            |                           0.00686 |
| (0, 1, 2, 3, 4)         |                           0.00704 |
| (0, 1, 4)               |                           0.09048 |
| (0, 1)                  |                           0.09091 |
| (0, 1, 3, 4)            |                           0.12305 |
| (0, 1, 3)               |                           0.12631 |
| (0, 2, 4)               |                           0.37836 |
| (0, 2)                  |                           0.42176 |
| (0, 4)                  |                           0.43317 |
| (0,)                    |                           0.48986 |
|